In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
#from model import DeepMALDRL
from tensorflow.keras.layers import Input, Dense, Lambda, concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
deepMAL = load_model('deep_mal_v5.h5')

In [8]:
print(deepMAL.summary())

Model: "DeepMAL-Packets"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1020, 32)          192       
                                                                 
 conv1d_1 (Conv1D)           (None, 1016, 64)          10304     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 127, 64)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 127, 200)          212000    
                                                                 
 flatten (Flatten)           (None, 25400)             0         
                                                   

In [87]:
deepMAL_result = pd.read_csv('datasets/predDF.csv', header=None)
deepMAL_input = pd.read_csv('datasets/malicious_data_generated_v5.csv', header=None)

In [88]:
print(deepMAL_input.head())
print(deepMAL_result.head())

   0     1     2     3     4     5     6     7     8     9     ...  1014  \
0   105    48    26    60    24    70     8   115     5    92  ...    42   
1   128    73   100   117     6   114    20   107   141    37  ...    18   
2    34    83    65   111   134    90    77   103    97   105  ...    69   
3    60   122   114     1    52   129    37    65    81    62  ...   147   
4   106    56    51   130   138   143   114    52    52    90  ...   124   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0   116    15   117    24   135    56    14   102    93  
1   110   102    38    66    44    38   109    12   144  
2    86    19    58   103   135    75   105   129    26  
3    88    31   131    27     6   102    59    48    92  
4    41    90     4    31    45    95     8    66    42  

[5 rows x 1024 columns]
          0
0  0.777735
1  0.989704
2  0.999327
3  0.956618
4  0.998270


In [89]:
percentage = 0.7
size = int(deepMAL_result.size*percentage)

deepMAL_result_train = deepMAL_result[0:size]
deepMAL_result_test = deepMAL_result[size:]
deepMAL_input_train = deepMAL_input[0:size]
deepMAL_input_test = deepMAL_input[size:]

In [90]:
early_stop = EarlyStopping(monitor = 'val_accuracy', patience = 5)
model_check = ModelCheckpoint(filepath='models/model.{epoch:02d}-{val_accuracy:.2f}.h5', monitor = "val_accuracy", verbose = 0, save_best_only = True, save_freq="epoch")

In [ ]:
#DRL_deepMAL = DeepMALDRL()

In [91]:
deepmal_packets_output = Input(shape=(1,1))

dense1 = Dense(32, activation='relu')(deepmal_packets_output)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)
dense4 = Dense(256, activation='relu')(dense3)
dense5 = Dense(512, activation='relu')(dense4)
dense6 = Dense(256, activation='relu')(dense5)
dense7 = Dense(128, activation='relu')(dense6)
dense8 = Dense(64, activation='relu')(dense7)
dense9 = Dense(32, activation='relu')(dense8)

output_layer = Dense(1024, activation='linear')(dense9)
drl_output = tf.reshape(output_layer, [-1,1024,1])

drl_model = Model(inputs=deepmal_packets_output, outputs=drl_output)

drl_model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [92]:
print(drl_model.summary())

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_60 (InputLayer)       [(None, 1, 1)]            0         
                                                                 
 dense_330 (Dense)           (None, 1, 32)             64        
                                                                 
 dense_331 (Dense)           (None, 1, 64)             2112      
                                                                 
 dense_332 (Dense)           (None, 1, 128)            8320      
                                                                 
 dense_333 (Dense)           (None, 1, 256)            33024     
                                                                 
 dense_334 (Dense)           (None, 1, 512)            131584    
                                                                 
 dense_335 (Dense)           (None, 1, 256)            1313

In [95]:
num_epochs = 100
BATCH_SIZE = 32

history = drl_model.fit(deepMAL_result_train, deepMAL_input_train, batch_size = BATCH_SIZE, epochs = num_epochs, shuffle = True, validation_data=(deepMAL_result_test, deepMAL_input_test), verbose = 2, callbacks = [early_stop, model_check])

Epoch 1/100
19/19 - 4s - loss: 7450.6245 - accuracy: 0.0066 - val_loss: 6917.5146 - val_accuracy: 0.0066 - 4s/epoch - 227ms/step
Epoch 2/100
19/19 - 0s - loss: 3960.0706 - accuracy: 0.0065 - val_loss: 2406.0112 - val_accuracy: 0.0066 - 427ms/epoch - 22ms/step
Epoch 3/100
19/19 - 0s - loss: 2082.5715 - accuracy: 0.0066 - val_loss: 2003.9563 - val_accuracy: 0.0066 - 407ms/epoch - 21ms/step
Epoch 4/100
19/19 - 0s - loss: 1945.9882 - accuracy: 0.0066 - val_loss: 1929.1766 - val_accuracy: 0.0066 - 419ms/epoch - 22ms/step
Epoch 5/100
19/19 - 0s - loss: 1923.6727 - accuracy: 0.0066 - val_loss: 1926.2002 - val_accuracy: 0.0066 - 427ms/epoch - 22ms/step
Epoch 6/100
19/19 - 0s - loss: 1921.7827 - accuracy: 0.0066 - val_loss: 1926.1489 - val_accuracy: 0.0066 - 411ms/epoch - 22ms/step
Epoch 7/100
19/19 - 0s - loss: 1921.4988 - accuracy: 0.0066 - val_loss: 1925.6017 - val_accuracy: 0.0066 - 461ms/epoch - 24ms/step


In [93]:
test = deepMAL_result.to_numpy()
result_test = deepMAL_input.to_numpy()
print(test)

print(test[0])

[[0.777735  ]
 [0.98970425]
 [0.99932706]
 [0.9566178 ]
 [0.9982695 ]
 [0.9999991 ]
 [0.719441  ]
 [0.9999921 ]
 [0.67101175]
 [0.9999901 ]
 [0.9778953 ]
 [0.9612642 ]
 [0.9999771 ]
 [0.9991713 ]
 [0.98728734]
 [0.99450564]
 [0.99995863]
 [0.99997777]
 [0.8593893 ]
 [0.99999464]
 [0.987004  ]
 [0.9999468 ]
 [0.999421  ]
 [0.95513153]
 [0.99994123]
 [0.6701166 ]
 [0.9948961 ]
 [0.99982923]
 [1.        ]
 [0.99593776]
 [0.99941206]
 [0.99652374]
 [0.98691815]
 [0.57881373]
 [0.9997023 ]
 [0.99999964]
 [0.8835265 ]
 [0.99763453]
 [0.99417835]
 [0.99753785]
 [0.99999565]
 [1.        ]
 [0.9523441 ]
 [0.9929542 ]
 [0.9232432 ]
 [0.8205538 ]
 [0.9854857 ]
 [0.98192644]
 [0.9991252 ]
 [0.99999726]
 [0.96453387]
 [0.99999964]
 [0.9999984 ]
 [0.99999845]
 [0.9998991 ]
 [0.9975569 ]
 [0.99996597]
 [0.80681807]
 [0.99994886]
 [0.99999964]
 [0.9961837 ]
 [0.9990594 ]
 [0.9989437 ]
 [0.7309162 ]
 [0.7964031 ]
 [0.9999506 ]
 [0.97269624]
 [0.9479393 ]
 [0.9951435 ]
 [0.60961396]
 [0.99743026]
 [0.99

In [94]:
print(np.shape(test))
reshaped = np.reshape(test[0], (1,1))
print(reshaped, np.shape(reshaped))

(856, 1)
[[0.777735]] (1, 1)


In [96]:
x = drl_model.predict(deepMAL_result.iloc[0])
print(x)

1/1 [==============================] - 1s 772ms/step
[[[65.311386]
  [62.341118]
  [62.60147 ]
  ...
  [67.961266]
  [64.43586 ]
  [63.924267]]]


In [97]:
print(deepMAL_result.iloc[0])
print(deepMAL_input.iloc[0])

0    0.777735
Name: 0, dtype: float64
0       105
1        48
2        26
3        60
4        24
       ... 
1019    135
1020     56
1021     14
1022    102
1023     93
Name: 0, Length: 1024, dtype: int64


In [73]:
print(np.shape(x))
print(np.shape(result_test[0]))

(1, 1, 1024)
(1024,)


In [63]:
print(result_test[0])

[128  73 100 ... 109  12 144]
